# CODIGO BASE

# Conexiones e importaciones

In [ ]:
# --- Librerías estándar de Python ---
import os                      # Gestionar variables de entorno y funciones del sistema operativo
import time                    # Controlar pausas y tiempos de espera en procesos o peticiones
from urllib.parse import quote # Codificar textos para incluirlos correctamente en URLs

# --- Librerías de terceros ---
import numpy as np             # Operaciones numéricas y manejo eficiente de arrays
import pandas as pd            # Manipulación y análisis de datos con DataFrames
import requests                # Realizar peticiones HTTP a APIs externas
import spotipy                 # Cliente oficial para interactuar con la API de Spotify
from dotenv import load_dotenv # Cargar variables de entorno desde un archivo .env
from spotipy.oauth2 import SpotifyClientCredentials  # Autenticación con Spotify mediante client_id y client_secret

# --- Conectores de bases de datos ---
import mysql.connector         # Conexión a bases de datos MySQL
from mysql.connector import Error  # Manejo de errores específicos de MySQL


In [ ]:
load_dotenv()  # Carga las variables definidas en el archivo .env en el entorno del sistema

# --- Spotify ---
mis_credenciales = SpotifyClientCredentials(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),      # Obtiene el client_id desde las variables de entorno
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET")  # Obtiene el client_secret desde las variables de entorno
)
spotify = spotipy.Spotify(auth_manager=mis_credenciales)  # Crea el cliente autenticado para usar la API de Spotify

# --- Last.fm ---
api_key_lastfm = os.getenv("API_KEY_LASTFM")            # API key de Last.fm cargada desde .env
shared_secret_lastfm = os.getenv("SHARED_SECRET_LASTFM")  # Shared secret de Last.fm cargado desde .env

# --- MySQL ---
MYSQL_HOST = os.getenv("MYSQL_HOST")          # Dirección del servidor MySQL
MYSQL_USER = os.getenv("MYSQL_USER")          # Usuario para la conexión MySQL
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")  # Contraseña del usuario MySQL


# Spotify

In [ ]:
def busqueda_spotify(generos, año):
    todas_las_canciones = []          # Lista donde guardaremos todas las canciones encontradas
    albumes_ya_vistos = set()         # Conjunto para evitar procesar el mismo álbum más de una vez

    print("Buscando canciones del año", año, "...\n")

    for genero in generos:            # Recorremos cada género solicitado
        print("Género:", genero)

        # Buscamos artistas asociados al género
        resultado_artistas = spotify.search(q="genre:" + genero, type="artist", limit=50)
        artistas = resultado_artistas["artists"]["items"]

        for artista in artistas:      # Recorremos cada artista encontrado
            nombre_artista = artista["name"]

            # Buscamos álbumes del artista filtrados por año
            busqueda = "artist:" + nombre_artista + " year:" + str(año)
            resultado_albumes = spotify.search(q=busqueda, type="album", limit=50)
            albumes = resultado_albumes["albums"]["items"]
            
            for album in albumes:     # Recorremos cada álbum encontrado
                id_album = album["id"]
                nombre_album = album["name"]
                
                if id_album in albumes_ya_vistos:  # Evitamos duplicados
                    continue
                
                albumes_ya_vistos.add(id_album)    # Marcamos el álbum como procesado

                # Obtenemos las canciones del álbum
                resultado_canciones = spotify.album_tracks(id_album)
                canciones = resultado_canciones["items"]

                for cancion in canciones:          # Recorremos cada canción del álbum
                    info = {
                        "nombre": cancion["name"], # Nombre de la canción
                        "artista": nombre_artista, # Artista principal
                        "album": nombre_album,     # Álbum al que pertenece
                        "genero": genero,          # Género buscado
                        "año": año                 # Año filtrado
                    }
                    todas_las_canciones.append(info)  # Guardamos la canción

        print(f"→ Canciones de {genero} añadidas\n")

    # --- Resumen final ---
    print("RESUMEN:")

    # Contamos cuántas canciones se obtuvieron por género
    for genero in generos:
        contador = sum(1 for c in todas_las_canciones if c["genero"] == genero)
        print(f"- {genero}: {contador} canciones")

    print("\nTotal de canciones encontradas:", len(todas_las_canciones))

    # Convertimos la lista de diccionarios en un DataFrame
    todas_las_canciones_df = pd.DataFrame(todas_las_canciones)
    return todas_las_canciones_df


In [ ]:
genero = ["country", "latin", "jazz", "rock"]   # Lista de géneros musicales que queremos analizar
año = 2018                                      # Año que queremos filtrar en la búsqueda (introduce aquí el año deseado)

# Llamamos a la función que consulta Spotify y devuelve un DataFrame con todas las canciones encontradas
canciones_año_df = busqueda_spotify(genero, año)


In [ ]:
canciones_año_df.to_csv("canciones_201X.csv", index=False)  # Exporta el DataFrame a un archivo CSV sin incluir la columna de índices


# Last FM

In [ ]:
# Verifica si la API Key de Last.fm está disponible en las variables de entorno
if not api_key_lastfm:
    print("ERROR: La variable de entorno 'API_KEY_LASTFM' no está configurada.")  # Mensaje de error si falta la clave
else:
    print("API Key de Last.fm cargada con éxito.")  # Confirmación si la clave se cargó correctamente


In [ ]:
url_last_fm = "http://ws.audioscrobbler.com/2.0/"  # URL base de la API de Last.fm para realizar peticiones


In [ ]:
def busqueda_info_artista(nombre_artista, api_key_lastfm):
    artista_codificado = quote(nombre_artista)  # Codifica espacios y caracteres especiales para que no rompan la URL
    
    url_last_fm = "http://ws.audioscrobbler.com/2.0/"  # URL base de la API de Last.fm
    params_info = {
        'method': 'artist.getinfo',   # Método de Last.fm para obtener información del artista
        'artist': artista_codificado, # Nombre del artista codificado para la URL
        'api_key': api_key_lastfm,    # Clave de la API cargada desde .env
        'format': 'json'              # Formato de respuesta deseado
    }

    try:
        # Realiza la petición HTTP con un timeout de 10 segundos
        response = requests.get(url_last_fm, params=params_info, timeout=10)
        response.raise_for_status()   # Lanza error si la respuesta no es 200 OK
        data = response.json()        # Convierte la respuesta a JSON

        if "artist" in data:          # Si Last.fm encontró al artista
            artista_info = data['artist']

            # Extrae el resumen de la biografía y elimina enlaces HTML
            bio_summary = artista_info.get('bio', {}).get('summary', '').split('<a href')[0].strip()

            return {
                'bio_resumen': bio_summary,  # Resumen limpio de la biografía
                'listeners': int(artista_info.get('stats', {}).get('listeners', 0)),  # Número de oyentes
            }

        else:
            # Caso en el que Last.fm no encuentra al artista
            return {'consulta_exitosa': False, 'error_lastfm': "No encontrado en Last.fm"}
        
    except requests.exceptions.RequestException as e:
        # Captura errores de red: HTTPError, Timeout, ConnectionError, etc.
        status_code = getattr(e.response, 'status_code', 'N/A')  # Extrae código HTTP si existe
        return {'consulta_exitosa': False, 'error_lastfm': f"Error API ({status_code}): {e}"}

    except Exception as e:
        # Captura cualquier otro error inesperado
        return {'consulta_exitosa': False, 'error_lastfm': f"Error Procesamiento: {e}"}


In [ ]:
datos_lastfm_df   # Muestra el DataFrame con la información obtenida de Last.fm para cada artista

In [ ]:
df_final.to_csv("anio_año_completo.csv", index=False)  # Exporta el DataFrame final a un archivo CSV sin incluir la columna de índices


# CREACION BBDD

In [ ]:
# Intentamos establecer la conexión con el servidor MySQL
try:
    cnx = mysql.connector.connect(
        host=MYSQL_HOST,        # Dirección del servidor MySQL
        user=MYSQL_USER,        # Usuario configurado en las variables de entorno
        password=MYSQL_PASSWORD # Contraseña del usuario
    )
    print('Conexión exitosa')   # Confirmación si la conexión se establece correctamente

except Error as e:
    print('Error al conectar:', e)  # Mensaje de error si la conexión falla


# Intentamos crear la base de datos si no existe
try:
    mycursor = cnx.cursor()  # Creamos un cursor para ejecutar comandos SQL
    query = "CREATE DATABASE IF NOT EXISTS MusicStream_db"  # SQL para crear la BD solo si no existe
    mycursor.execute(query)  # Ejecutamos la consulta
    print("Query exitosa")   # Confirmación de que la BD fue creada o ya existía

except:
    print("Error.")          # Mensaje genérico si ocurre algún problema al ejecutar la query


In [ ]:
mycursor.execute("USE MusicStream_db")  # Selecciona la base de datos donde se crearán las tablas

# Nota: se usan varios .execute() porque cada uno ejecuta una instrucción SQL independiente.
# Si todas se llamaran "query" y solo se ejecutara la última, las anteriores se perderían.

query = '''CREATE TABLE IF NOT EXISTS Canciones(
    ID_Cancion INT AUTO_INCREMENT,   -- Identificador único de cada canción
    Nombre TEXT NOT NULL,            -- Nombre de la canción
    Artista VARCHAR(45),             -- Nombre del artista
    Album TEXT,                      -- Álbum al que pertenece
    Genero VARCHAR(30),              -- Género musical
    Lanzamiento YEAR,                -- Año de lanzamiento
    Info TEXT,                       -- Información adicional (bio, resumen, etc.)
    Oyentes INT,                     -- Número de oyentes (Last.fm)
    PRIMARY KEY (ID_Cancion)         -- Clave primaria de la tabla
);'''

mycursor.execute(query)  # Crea la tabla si no existe


In [ ]:
df_completo = pd.read_csv("CSV/TABLA_FINAL.csv")  # Carga el archivo CSV final en un DataFrame para su análisis o procesamiento

In [ ]:
# Seleccionamos la base de datos donde insertaremos los registros
mycursor.execute("USE MusicStream_db")

# Query de inserción con placeholders (%s) para cada columna
query_insert = """
INSERT INTO Canciones (Nombre, Artista, Album, Genero, Lanzamiento, Info, Oyentes) 
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

try:
    # Reemplazamos valores nulos (NaN) por None para que MySQL los acepte correctamente
    df_corregido_completo = df_completo.replace({np.nan: None, 'nan': None, 'NaN': None})
    
    # Convertimos el DataFrame a una lista de listas para ejecutemany()
    datos = df_corregido_completo.values.tolist()

    # Inserta todos los registros en la tabla de una sola vez
    mycursor.executemany(query_insert, datos)
    
    print(f"{mycursor.rowcount} registros insertados")  # Muestra cuántas filas se insertaron

    cnx.commit()  # Confirma los cambios en la base de datos (obligatorio para guardar la inserción)

except Error as e:
    print("Error al insertar los datos:", e)  # Mensaje detallado del error
    cnx.rollback()  # Revierte cualquier cambio si ocurre un error durante la inserción


In [ ]:
cnx.close()  # Cierra la conexión con el servidor MySQL para liberar recursos